# Project Team: Walking in Paris

### Objective:
Identify the attributes that make Airbnb “Successful” 
Find the best place to stay in Hawaii

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time

# Import statistic library
import scipy.stats as stats
from scipy.stats import linregress

# Import seaborn library to create figure
import seaborn as sns

import pprint
import json
from time import sleep
from datetime import date
import zipfile

# Incorporated citipy to determine city based on latitude and longitude
# pip install citipy
from citipy import citipy

# Output File location
output_folder = "output_data"

OPEN THE DATASET
Open the dataset from the zip file and ensure memory resources

In [ ]:
# Open the zip file
zf = zipfile.ZipFile('./source_data/detail_listings.zip') 
df = pd.read_csv(zf.open('detail_listings.csv'), low_memory=False)

rows_in_df = len(df)
print(f'Data set has {rows_in_df} rows')

DATA PREPARATION STEP 1:
Organize the columns to make the analysis easy

In [ ]:
# Group the columns and show the list of the columns

print('')
print('----------------------')
print('Property-specific columns:')
print('----------------------')

# Create a wildcard to search the terms
cols = ["experiences_offered", "street", "city", "state", "zipcode", "market", "country", "property"
    , "location", "accomodate", "room", "guest", "is_business_travel_ready", "instant_bookable", "cancel"]

# Lopps the columns
for col in df.columns: 
    if col in cols:
        print(col) 
        
print('')
print('----------------------')
print('All columns:')
print('----------------------')
for col in df.columns: 
    print(col)

DATA PREPARATION STEP 2: 
Created separate data dictionary data frame to allow us to “group” columns (Property-related, Neighborhood-related, etc)

In [ ]:
# Create a dataframe with one (1) row for each column in the source data:
data_dict = pd.DataFrame(columns = [
    "ColumNumber"
    , "Column"
    , "Group"
    , "Subgroup"
    , "Notes"
])

# Loop through the columns in the data frame
for column in df:
    data_dict = data_dict.append({
            "Column": column
            , "Group": ""
            , "Subgroup": ""
            , "Notes": ""
        }
        , ignore_index=True
    )

# Make the index the column number
data_dict['ColumNumber'] = data_dict.index
        
# Add context for each column 
for index, row in data_dict.iterrows():
    val = str(row["Column"])
    
    if val == "id":
        data_dict.loc[index, "Notes"] = "Primary Key"
    # Val.startswith("host_")  
    if val.find("host") > -1: 
        data_dict.loc[index, "Group"] = "Host-related"
    
    if val.find("review") > -1:
        data_dict.loc[index, "Group"] = "Review-related"
        
    if (val.find("rule") > -1) | (val.find("require") > -1):
        data_dict.loc[index, "Group"] = "Requirements-related"
        
    if (val.find("neighbo") > -1):
        data_dict.loc[index, "Group"] = "Neighborhood-related"
        
    if (val.find("nights") > -1):
        data_dict.loc[index, "Group"] = "Booking-related"
        
    # Property specific:
    cols = [
        "property", "room", "guest", "is_business_travel_ready", "instant_bookable", "cancel",
        "space", "name", "summary", "description", "notes", "transit", "access", "interaction",
        "property_type", "room_type", "amenities"
    ]
    if any(word in val for word in cols):
        data_dict.loc[index, "Group"] = "Property-related"
        
    # Property - "Location" specific:
    cols = [
        "location", "street", "city", "state", "zipcode", "market", "country"
        , "latitude", "longitude", "is_location_exact"
    ]
    if any(word in val for word in cols):
        data_dict.loc[index, "Group"] = "Property-related"
        data_dict.loc[index, "Subgroup"] = "Location"
        
    # Property - "Interior" specific:
    cols = [
        "bathrooms", "bedrooms", "beds", "square_feet"
    ]
    if any(word in val for word in cols):
        data_dict.loc[index, "Group"] = "Property-related"
        data_dict.loc[index, "Subgroup"] = "Interior"
        
    # Property - "Pricing" specific:
    # Variables related to price, or that we might associate with price
    cols = [
        "price", "weekly_price", "monthly_price"        
        , "security_deposit", "cleaning_fee"
        , "accomodates", "guests_included", "extra_people"
        , "experiences_offered"
    ]
    if any(word in val for word in cols):
        data_dict.loc[index, "Group"] = "Property-related"
        data_dict.loc[index, "Subgroup"] = "Pricing"
        
    # Accomodate a group and subgroup of properties:
    if val == "accommodates":
        data_dict.loc[index, "Group"] = "Property-related"
        data_dict.loc[index, "Subgroup"] = "Pricing"

print(data_dict.groupby(["Group", "Subgroup"])["Column"].count())
print('')
print('Columns without a group:')
data_dict[data_dict["Group"] == ""]

DATA PREPARATION STEP 3:
- Create bin to separate the host by the number of properties 

In [ ]:
# Create the bins to separate the number of properties by host
bins = [0, 50, 100, 150, 200, 250, 299]

# labels - always one less than the range
group_names = ["0-50 properties", "51-100 properties", "101-150 properties", "151-200 properties",\
               "201-250 properties", "251+ properties"]

df["Bin_NumProperties"] = pd.cut(df["calculated_host_listings_count"], bins, labels=group_names)


# Loop through and get aggs:
total_reviews_per_host = df.groupby("host_id").sum()["number_of_reviews"]
average_review_per_host = df.groupby("host_id").mean()["review_scores_rating"]
rating_mean_by_bin = df.groupby("Bin_NumProperties").mean()["review_scores_rating"]

#df_analyze["rating_value_mean"] = df_analyze["review_scores_value"].mean()
df.head(1)

DATA PREPARATION STEP 4:
Create a group by neighbourhood and number of reviews


In [ ]:
# data_dict[data_dict["Group"] == "Host-related"]
host_group = df.groupby("neighbourhood").agg({
    "number_of_reviews": [
        np.count_nonzero,
        np.mean, 
        np.median, 
        np.var, 
        np.std
    ]
})

host_group.rename(columns={'count_nonzero':'Reviews'})
host_group.head()
# bin on reviews_per_month

DATA PREPARATION STEP 5:
- Create bin to separate by the number of reviews

In [ ]:
bins_num_reviews = [10, 20, 50, 100, 200, 300, 350, 400, 450, 500, 550, 600, 100000]

# labels - always one less than the range
group_names_num_reviews = [
    "1-10"
    , "11-20"
    , "21-50"
    , "100-200"
    , "201-250"
    , "251-300"
    , "301-350"
    , "351-400"
    , "401-550"
    , "451-500"
    , "501-550"
    , "551-600"
]

df["Bin_NumReviews"] = pd.cut(
    df["number_of_reviews"], 
    bins = bins_num_reviews, 
    labels = group_names_num_reviews
)

df["Bin_NumReviews"].value_counts()


In [ ]:
# prin the number of reviews
print(f'Sum: {df["number_of_reviews"].sum()}')
print(f'Count: {df["number_of_reviews"].count()}')

df.groupby("Bin_NumReviews")["number_of_reviews"].sum()

In [ ]:
# Aggregate the group by numebr of reviews
df.groupby("Bin_NumReviews").agg({
    "number_of_reviews": [
        np.count_nonzero,
        np.mean, 
        np.median, 
        np.var, 
        np.std
    ]
})

DATA VALIDATION 1:
Create a figure to visualize neighbourhood and number of reviews

In [ ]:
# Use seaborn library to create a boxplot
# sns.boxplot(x="day", y="total_bill", hue="smoker", data=df, palette="Set1")
sns.boxplot(
    x = "neighbourhood_group_cleansed",
    y = "number_of_reviews",
    #hue = "price",
    data = df,
    palette = "Set1"
)


DATA VALIDATION 2:
Evaluate dataset to identify variables that are correlated

In [ ]:
corr = df.corr()
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=np.bool))
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))
# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(
    corr,
    mask=mask,
    cmap=cmap,
    vmax=.3,
    center=0,
    square=True,
    linewidths=.5,
    cbar_kws={"shrink": .5}
)

In [ ]:
df.loc[:,data_dict[data_dict["Group"] == "Host-related"]["Column"]]

CLEANING THE DATASET 1:
Cleaning raw data to delete the column with not values, columns values duplicated, columns with data not required.

In [ ]:
# Drop unneeded columns:
df = df.drop(columns = [
    "listing_url", 
    "thumbnail_url", 
    "picture_url",    
    "medium_url",    
    "xl_picture_url", 
    "host_url",       
    "host_thumbnail_url", 
    "host_picture_url",  
    "scrape_id",   
    "host_listings_count", 
    "host_acceptance_rate", 
    "calendar_last_scraped",
    "bed_type",
    "last_scraped",
    "calendar_updated",
    "has_availability",
    "availability_30", 
    "availability_60", 
    "availability_90", 
    "license"
])

df.head(1)

CLEANING THE DATASET 2:
- Get names of indexes for which column number_of_reviews has value 0

In [ ]:
# Drop by index:
indexNames = df[df['number_of_reviews'] == 0 ].index
df.drop(indexNames , inplace=True)

# Drop rows with number_of_reviews = NaN
df.dropna(axis=0, subset=('number_of_reviews', ))

# Drop rows with NaN review data
df.dropna(axis=0, subset=(
    'review_scores_rating', 
    'review_scores_accuracy', 
    'review_scores_cleanliness', 
    'review_scores_checkin', 
    'review_scores_communication', 
    'review_scores_location', 
    'review_scores_value'
))

print(f'Remaining rows after cleanup: {len(df)}')


CLEANING THE DATASET FROM AIRBNB STEP 3:
- Remove the dollar symbol ($) from the pricing  based columns:
"price", "weekly_price","monthly_price","security_deposit","cleaning_fee", "extra_people"  

In [ ]:
# Strip the $ out of the pricing-based columns
data_dict[(data_dict["Group"] == "Property-related") & (data_dict["Subgroup"] == "Pricing")]

if df["price"].dtype != "float64":
    df["price"] = df["price"].str.replace("$", "")
    df["price"] = df["price"].str.replace(",", "")
    df["price"] = df["price"].astype("float64")

if df["weekly_price"].dtype != "float64":
    df["weekly_price"] = df["weekly_price"].str.replace("$", "")
    df["weekly_price"] = df["weekly_price"].str.replace(",", "")
    df["weekly_price"] = df["weekly_price"].astype("float64")

if df["monthly_price"].dtype != "float64":
    df["monthly_price"] = df["monthly_price"].str.replace("$", "")
    df["monthly_price"] = df["monthly_price"].str.replace(",", "")
    df["monthly_price"] = df["monthly_price"].astype("float64")
    
if df["security_deposit"].dtype != "float64":
    df["security_deposit"] = df["security_deposit"].str.replace("$", "")
    df["security_deposit"] = df["security_deposit"].str.replace(",", "")
    df["security_deposit"] = df["security_deposit"].astype("float64")
    
if df["cleaning_fee"].dtype != "float64":
    df["cleaning_fee"] = df["cleaning_fee"].str.replace("$", "")
    df["cleaning_fee"] = df["cleaning_fee"].str.replace(",", "")
    df["cleaning_fee"] = df["cleaning_fee"].astype("float64")   

if df["extra_people"].dtype != "float64":
    df["extra_people"] = df["extra_people"].str.replace("$", "")
    df["extra_people"] = df["extra_people"].str.replace(",", "")
    df["extra_people"] = df["extra_people"].astype("float64")
    
data_dict


HYPOTHESIS
-----------------------------------

Ho:
 There are certain factors that have an impact on the overall rating of an Air BNB property.
 
H1:
 None of the factors that we are analyzing have any impact on the overall rating of a property.


ANALYSIS OF THE DATA
-----------------------------------

Evaluate the relationship between "how many properties someone has" and review_scores_rating
-----------------------------------------------------------------------------------------------------------------

In [ ]:
# Define the bins by number of properties by host
bins = [0, 50, 100, 150, 200, 250, 299]

# labels - always one less than the range
group_names = ["0-50 properties", "51-100 properties", "101-150 properties", "151-200 properties",\
               "201-250 properties", "251+ properties"]

df["Bin_NumProperties"] = pd.cut(df["calculated_host_listings_count"], bins, labels=group_names)

rating_mean_by_bin = df.groupby("Bin_NumProperties").mean()["review_scores_rating"]

#df.head(2)
rating_mean_by_bin

In [ ]:
# Define the axis
x_axis = group_names
y_axis = rating_mean_by_bin

# Bring in the colors from Hawaii
hawaiian_hibiscus_hex_palette = ["#ffa4cd", "#ff4e9e", "#ff0040", "#ffcd55", "#ffa700"]

# Define the properties of the figure
plt.subplots(figsize=(15,10))
plt.title("Average Rating per Number of Properties")
plt.xlabel("Number of Properties")
plt.ylabel("Average Rating")
plt.xticks(rotation=90)
plt.ylim(85, 100)
plt.bar(x_axis, y_axis, color = hawaiian_hibiscus_hex_palette)

# Show the figure
plt.show

In [ ]:
df.boxplot("review_scores_rating", by="Bin_NumProperties", figsize=(20, 10))

In [ ]:
# creating a set of group by Number of properties and review score rating

# Extract individual groups
group1 = df[df["Bin_NumProperties"] == "0-50 properties"]["review_scores_rating"].dropna()
group2 = df[df["Bin_NumProperties"] == "51-100 properties"]["review_scores_rating"].dropna()
group3 = df[df["Bin_NumProperties"] == "101-150 properties"]["review_scores_rating"].dropna()
group4 = df[df["Bin_NumProperties"] == "151-200 properties"]["review_scores_rating"].dropna()
group5 = df[df["Bin_NumProperties"] == "201-250 properties"]["review_scores_rating"].dropna()
group6 = df[df["Bin_NumProperties"] == "251+ properties"]["review_scores_rating"].dropna()

stats.f_oneway(group1, group2, group3, group4, group5, group6)

In [ ]:
neighborhood_list = df["neighbourhood"].unique()

average_rating_per_neighborhood = df.groupby("neighbourhood").mean()["review_scores_rating"]
average_rating_per_neighborhood

x_axis = neighborhood_list
y_axis = average_rating_per_neighborhood

cleaned_neighborhood_list = [x for x in neighborhood_list if str(x) != 'nan']
cleaned_neighborhood_list.sort()
cleaned_neighborhood_list

x_axis = cleaned_neighborhood_list
y_axis = average_rating_per_neighborhood

hawaiian_hibiscus_hex_palette = ["#ffa4cd", "#ff4e9e", "#ff0040", "#ffcd55", "#ffa700"]

plt.subplots(figsize=(15,10))
plt.xticks(rotation=90)
plt.title("Average Rating per Number of Properties")
plt.xlabel("Number of Properties")
plt.ylabel("Average Rating")
plt.ylim (65, 100)
plt.bar(x_axis, y_axis, color = hawaiian_hibiscus_hex_palette, align='center')
plt.show

In [ ]:
# Define axis
x_axis = df["price"]
y_axis = df["review_scores_rating"]


mask = ~np.isnan(x_axis) & ~np.isnan(y_axis)
slope, intercept, r_value, p_value, std_err = stats.linregress(x_axis[mask], y_axis[mask])
regress_values = x_axis * slope + intercept
line_eq = "y = " + str(round(slope,4)) + "x + " + str(round(intercept,4))

plt.subplots(figsize=(15,10))
plt.xticks(rotation=90)
plt.xlim(0, 5500)
plt.plot(x_axis,regress_values,"r-")
plt.scatter(x_axis,y_axis)
plt.annotate(line_eq,(4000,50),fontsize=15,color="red")
plt.xlabel('Price')
plt.ylabel('Rating')
print(f"The r-squared is: {r_value}")
plt.show()

In [ ]:
# create a dataframe with no 0's for 'number of beds' becuase that would be inaccurate data
df2 = df.copy()
df2 = df2[(df2['beds'] != 0)]

In [ ]:
# zero beds is actually an option. A 'sofa bed' is not a real bed and is not counted as such.
bins = [0, 2, 4, 6, 8, 10, 12, 20]

group_names = ["1-2 beds", "3-4 beds", "5-6 beds", "7-8 beds", "9-10 beds", "11-12 beds", "13+ beds"]

df2["bin_bed_count"] = pd.cut(df2["beds"], bins, labels=group_names)

rating_mean_by_bed_count = df2.groupby("bin_bed_count").mean()["review_scores_rating"]

In [ ]:
x_axis = group_names
y_axis = rating_mean_by_bed_count

hawaiian_hex_palette = ["#00739f", "#b23948", "#ff000e", "#ff407c", "#ffb2e1"]

plt.subplots(figsize=(15,10))
plt.title("Average Rating per Number of Beds")
plt.xlabel("Number of Beds")
plt.ylabel("Average Rating")
plt.xticks(rotation=90)
plt.ylim(0, 100)
plt.bar(x_axis, y_axis, color = hawaiian_hibiscus_hex_palette)
plt.show

In [ ]:
# Bin by island
isle = df[['latitude', 'longitude']].head(50).copy()
isle["Island"] = ""

curr_rows = len(isle)
print(f'There are {curr_rows} coordinates in the dataframe')

df["neighbourhood_group_cleansed"]

In [ ]:
average_rating_island = df.groupby("neighbourhood_group_cleansed").mean()["review_scores_rating"]
properties_island = df.groupby("neighbourhood_group_cleansed").count()["id"]
average_rating_island.head()

In [ ]:
# set x and y axis for bar graph
x_axis = df["neighbourhood_group_cleansed"].unique()
y_axis = average_rating_island

# Define the properties of the figure
plt.subplots(figsize=(15,10))

# print lables/title and x/y limits to improve visual
plt.title("Average Rating per Island")
plt.xlabel("Islands")
plt.ylabel("Average Rating")
plt.xticks(rotation=90)
plt.ylim(85, 100)

# define color scheme
hawaiian_hibiscus_hex_palette = ["#FFA4CD", "#FF4E9E", "#FF0040", "#FFCD55", "#FFA700"]

# display graph
plt.bar(x_axis, y_axis, color = hawaiian_hibiscus_hex_palette)
plt.show

In [ ]:
df.boxplot("review_scores_rating", by="neighbourhood_group_cleansed", figsize=(20, 10))

In [ ]:
# Define a group by every rating properties
average_rating_prop = df.groupby("room_type").mean()["review_scores_rating"]
#property_type = df.groupby("property_type").count()["id"]

# dropped NaN values for property types that do not have ratings in hawaii such as 'igloo'
average_rating_prop = average_rating_prop.dropna()
average_rating_prop

In [ ]:
# set x and y axis for bar graph
x_axis = df["room_type"].unique()
y_axis = average_rating_prop

# print lables/title and x/y limits to improve visual
plt.subplots(figsize=(15,10))
plt.title("Average Rating per Type of Property")
plt.xlabel("Room Type")
plt.ylabel("Average Rating")
plt.xticks(rotation=90)
plt.ylim(85, 100)

# display graph
plt.bar(x_axis, y_axis, color = hawaiian_hibiscus_hex_palette)
plt.show

In [ ]:
df.boxplot("review_scores_rating", by="room_type", figsize=(20, 10))

In [ ]:
# Build a pie to show the availability by property type in Airbnb Hawaii.
prop_type_pie=df.property_type.value_counts()/len(df.property_type)
prop_type_pie.plot.pie(autopct='%.2f',fontsize=12,figsize=(8,8))
plt.title('Property types availability in AirBnB',fontsize=20)

In [ ]:
# Build a pie to show the availability by property room type in Airbnb Hawaii.
room_type_pie=df.room_type.value_counts()/len(df.room_type)
room_type_pie.plot.pie(autopct='%.2f',fontsize=12,figsize=(8,8))
plt.title('Property Room Types Availability in AirBnB',fontsize=20)

In [ ]:
plt.subplots(figsize=(15,10))
df.groupby(['neighbourhood_group_cleansed','room_type']).room_type.count().plot.barh(stacked=True)
plt.ylabel('Room Types per Island')
plt.xlabel('Average Ratings')
plt.title('Neighbourhood groups Vs Room Types')

In [ ]:
plt.subplots(figsize=(15,10))
df.groupby(['neighbourhood_group_cleansed','room_type']).review_scores_rating.mean().plot.barh(stacked=True)
plt.ylabel('Room Types per Island')
plt.xlabel('Number of Rooms')
plt.title('Neighbourhood groups Vs Room Types Availability')


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df.longitude, df.latitude, c=df["availability_365"] , cmap='spring',\
            edgecolor='black', linewidth=1, alpha=0.75)

cbar = plt.colorbar()
cbar.set_label('availability_365')

In [ ]:
df["availability_365"]

# 